In [ ]:
import pandas as pd

In [ ]:
# Import data and clean up dataframe
data = pd.read_csv('iris.txt', sep = ',', header = None)
data.columns = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width', 'Species']
data.at[34, 'Petal Width'] = 0.2
data.at[37, 'Sepal Width'] = 3.6
data.at[37, 'Petal Length'] = 1.4